# Imports

In [1]:
import numpy as np
import pandas as pd

from itertools import combinations

# 1. Get Rid of the Loops

### 1.1. Eliminate Loops with List Comprehension, Map Function, & itertools

In [2]:
# List of HP, Attack, Defense, Speed
poke_stats = [
    [90, 92, 75, 60],
    [25, 20, 15, 90],
    [65, 130, 60, 75],
]

In [3]:
%%timeit
totals = []
for row in poke_stats:
    totals.append(sum(row))

584 ns ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [4]:
%%timeit
totals_comp = [sum(row) for row in poke_stats]

553 ns ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
%%timeit
totals_map = [*map(sum, poke_stats)]

537 ns ± 9.87 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
poke_types = ['Bug', 'Fire', 'Ghost', 'Grass', 'Water']

In [7]:
%%timeit
# Nested for loop approach
combos = []
for x in poke_types:
    for y in poke_types:
        if x == y:
            continue
    if ((x,y) not in combos) & ((y,x) not in combos):
        combos.append((x,y))

2.81 µs ± 74 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
%%timeit
# Built-in module approach
combos2 = [*combinations(poke_types, 2)]

594 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### 1.2. Eliminate Loops with NumPy

In [9]:
%%timeit
avgs = []
for row in poke_stats:
    avg = np.mean(row)
    avgs.append(avg)

32 µs ± 453 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%%timeit 
avgs = np.array(poke_stats).mean(axis=1)

12.6 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# 2. Writing Better Loops

### 2.1. Moving calculations above a loop

In [11]:
%%timeit -n 1 -r 1

names = ['Absol', 'Aron', 'Jynx', 'Natu', 'Onix']
attacks = np.array([130, 70, 50, 50, 45])

for pokemon,attack in zip(names, attacks):
    total_attack_avg = attacks.mean()
    if attack > total_attack_avg:
        print(
            "{}'s attack: {} > average: {}!"
            .format(pokemon, attack, total_attack_avg)
        )

Absol's attack: 130 > average: 69.0!
Aron's attack: 70 > average: 69.0!
286 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%%timeit -n 1 -r 1

names = ['Absol', 'Aron', 'Jynx', 'Natu', 'Onix']
attacks = np.array([130, 70, 50, 50, 45])

# Calculate total average once (outside the loop)
total_attack_avg = attacks.mean()

for pokemon,attack in zip(names, attacks):
    if attack > total_attack_avg:
        print(
            "{}'s attack: {} > average: {}!"
            .format(pokemon, attack, total_attack_avg)
        )

Absol's attack: 130 > average: 69.0!
Aron's attack: 70 > average: 69.0!
303 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 2.2. Holistic Conversions

In [13]:
# Data for example

pokemon = pd.read_csv('data/pokemon.csv')
names_list = pokemon['Name']
legend_status_list = pokemon['Legendary']
generations_list = pokemon['Generation']

In [14]:
%%timeit

poke_data = []
for poke_tuple in zip(names_list, legend_status_list, generations_list):
    poke_list = list(poke_tuple)
    poke_data.append(poke_list)

390 µs ± 7.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


However, converting each tuple to a list within the loop is not very efficient. Instead, we should collect all of our poke_tuples together, and use the map function to convert each tuple to a list. The loop no longer converts tuples to lists with each iteration. Instead, we moved this tuple to list conversion outside (or below) the loop. That way, we convert data types all at once (or holistically) rather than converting in each iteration.

In [15]:
%%timeit

poke_data_tuples = []
for poke_tuple in zip(names_list, legend_status_list, generations_list):
    poke_data_tuples.append(poke_tuple)
    
poke_data = [*map(list, poke_data_tuples)]

377 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
